In [2]:
import os
import re
import time
import socket

In [3]:
# 크롤링 결과를 저장하는 파일
CRAWLED_RESULT_SAVED_FILE = "res/crawl_data/result.txt"

In [4]:
def record_url_info_thread():
    while (True):
        f = open(CRAWLED_RESULT_SAVED_FILE, 'w')
        print(f.write('test'))
        f.close()

In [5]:
#record_info()

4


In [8]:
!dir res\crawl_data

 C 드라이브의 볼륨: 오후반_OS
 볼륨 일련 번호: E808-E5FC

 C:\Users\user2\python_works\PythonWorkshop-1\res\crawl_data 디렉터리

2021-10-21  오후 03:45    <DIR>          .
2021-10-21  오후 03:45    <DIR>          ..
2021-10-21  오후 03:45                 4 result.txt
               1개 파일                   4 바이트
               2개 디렉터리  147,387,457,536 바이트 남음


In [9]:
!type res\crawl_data\result.txt

test


In [10]:
from flask import Flask, url_for, request, jsonify, redirect, Response
from flask_cors import CORS, cross_origin

import json
import requests

In [11]:
import selenium
from selenium import webdriver

In [51]:
# 키보드 입력 처리
from selenium.webdriver.common.keys import Keys

In [52]:
def cont_youtube_crawl_thread(text):
    #url = "https://www.youtube.com/"
    url = "https://www.youtube.com/results?search_query="

    driver = webdriver.Chrome(r"C:\chromedriver.exe")
    
    search_url = url + text
    driver.get(url = search_url)
    
    # body 정보 확보
    body = driver.find_element_by_tag_name('body')
    
    # URL 확보
    allContents = driver.find_elements_by_css_selector(
        "#contents > ytd-item-section-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
    )

    hrefURL = []

    for item in allContents:
        if (item != ''):
            hrefURL.append(item.get_attribute('href'))
            
    shelfContents = driver.find_elements_by_css_selector(
        "#contents > ytd-shelf-renderer > div > div > ytd-vertical-list-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
    )
    
    for item in shelfContents:
        if (item != ''):
            hrefURL.append(item.get_attribute('href'))

    print(hrefURL)
    print("크롤링 개수: ", len(hrefURL))
    
    while (True):
        body.send_keys(Keys.PAGE_DOWN)
    
    # y축으로 3000 픽셀 이동함
    #driver.execute_script("window.scrollTo(0, 3000)")

In [53]:
# 현재 상황에서 기록하는 스레드와 크롤링 하는 두 개의 스레드가 있음
# 여기에서 어떤 문제점이 발생할 수 있을까 ?

# 크롤링하는 스레드 - 검색 키워드를 기반으로 크롤링
# 기록하는 스레드 - 크롤링한 URL 저장

In [54]:
cont_youtube_crawl_thread("시고르자브종")

['https://www.youtube.com/watch?v=PJ5QuJluIWU', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/watch?v=UUtnh3J8xfQ', 'https://www.youtube.com/watch?v=SIRv-UeJZXM', 'https://www.youtube.com/watch?v=9KnTJVQRYag', 'https://www.youtube.com/watch?v=OMBjyeSJLqY', 'https://www.youtube.com/watch?v=z48eJfp_HP4', 'https://www.youtube.com/watch?v=gmd4OnrxR0Q', 'https://www.youtube.com/watch?v=J1pa4irZwEM', 'https://www.youtube.com/watch?v=ePSG2dJvx48', 'https://www.youtube.com/watch?v=nfbGqacJtmw', 'https://www.youtube.com/watch?v=o0HDOupqRi4', 'https://www.youtube.com/watch?v=22_1CRL35Qk', 'https://www.youtube.com/watch?v=-XYSZpWvSGs', 'https://www.youtube.com/watch?v=5aNHqVXQ4W8', 'https://www.youtube.com/watch?v=JLkAv6kwGJw', 'https://www.youtube.com/watch?v=ktK-8XU8UzI', 'https://www.youtube.com/watch?v=7IkGv9asbkQ', 'https://www.youtube.com/watch?v=BgHNCRSYH5o', 'https://www.youtube.com/watch?v=H_R_SvQTw2s', 'https://www.youtube.com/watch?v=-lgYmZsFK-U', 'https://www

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=94.0.4606.81)
